In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 3.0 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.5-cp310-cp310-linux_x86_64.whl size=293378 sha256=4a6089635a93a98821e4da86f269b94e3701def53ac0d04c8c3126ef6966683c
  Stored in directory: /root/.cache/pip/wheels/06/80/9b/98db0d58349a2f5c09f8406789ade4270762f97b7d26f2fa22
Successfully built peewee


Automated Trading Strategy: Brainstorming Ideas

In [34]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

def prepare_data(ticker, interval="1d", period="5y", window_size=5):
    # Preprocessing the data.
    data = yf.download(ticker, interval=interval, period=period)
    data["Percentage Change"] = (data["Close"] / data["Open"] - 1) * 100
    
    # Getting features out of it.
    features, labels = [], [] 
    filtered_data = data["Percentage Change"].tolist()
    filtered_data = np.array(filtered_data).reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    filtered_data = scaler.fit_transform(filtered_data)
    print(filtered_data)
    for index in range(window_size, len(filtered_data)):
        labels.append(filtered_data[index])
        features.append(filtered_data[index - window_size: index])
    features, labels = np.array(features), np.array(labels)
    return features, labels

def distance_function(x1, x2):
    distance, multiplier = 0, 1
    for index in range(len(x1)):
        distance += multiplier * (x1[index] - x2[index])
        multiplier += 1
    return distance

def final_model_testing(features, labels, k_optimal=5):
    split_index = len(features) - len(features) // 10
    X_train, X_test, y_train, y_test = features[:split_index], features[split_index:], labels[:split_index], labels[split_index:]
    knn = KNeighborsClassifier(n_neighbors=k_optimal, metric=distance_function)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    print(f"Accuracy = {accuracy_score(y_pred, y_test)}")

features, labels = prepare_data("RELIANCE.NS")
features = np.reshape(features, (features.shape[0], features.shape[1], 1))

[*********************100%%**********************]  1 of 1 completed

[[-0.20793159]
 [-0.05702983]
 [-0.09241034]
 ...
 [-0.07739973]
 [ 0.02627098]
 [-0.12172931]]


In [42]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import plotly.graph_objects as go

def prepare_data(ticker, col, interval="1d", period="5y", window_size=5):
    # Downloading data
    data = yf.download(ticker, interval=interval, period=period)

    # Preparing features and labels
    features, labels = [], []
    filtered_data = data[col].values
    filtered_data = np.array(filtered_data).reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    filtered_data = scaler.fit_transform(filtered_data)
    
    for index in range(window_size, len(filtered_data)):
        features.append(filtered_data[index - window_size: index])
        labels.append(filtered_data[index])
    
    features, labels = np.array(features), np.array(labels)
    return features, labels, scaler

def lstm_implementation(features, labels, window_size=5):
    # Splitting data into training and testing sets
    split_index = len(features) - len(features) // 10
    X_train, X_test = features[:split_index], features[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]

    # Building the model
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=(window_size, 1)))
    model.add(LSTM(units=32, return_sequences=True))
    model.add(LSTM(units=16))
    model.add(Dense(units=1))
    model.compile(optimizer="adam", loss="mean_squared_error")
    
    # Training the model
    model.fit(X_train, y_train, epochs=10, batch_size=10)
    
    # Making predictions
    y_pred = model.predict(X_test)
    
    return y_test, y_pred

# Preparing the data
features_close, labels_close, scaler_close = prepare_data("RELIANCE.NS", "Close")
features_close = np.reshape(features_close, (features_close.shape[0], features_close.shape[1], 1))
features_open, labels_open, scaler_open = prepare_data("RELIANCE.NS", "Open")
features_open = np.reshape(features_open, (features_open.shape[0], features_open.shape[1], 1))

# Running the LSTM implementation
y_test_close, y_pred_close = lstm_implementation(features_close, labels_close)
y_test_open, y_pred_open = lstm_implementation(features_open, labels_open)

# Inverse scaling for interpretation
y_test_close = scaler_close.inverse_transform(y_test_close)
y_pred_close = scaler_close.inverse_transform(y_pred_close)
y_test_open = scaler_open.inverse_transform(y_test_open)
y_pred_open = scaler_open.inverse_transform(y_pred_open)

# Plotting the results
fig = go.Figure()
delta_predicted = y_pred_close - y_pred_open
delta_actual = y_test_close - y_test_open
fig.add_trace(go.Scatter(y=delta_actual.flatten(), mode="lines", name="Actual Change in Price"))
fig.add_trace(go.Scatter(y=delta_predicted.flatten(), mode="lines", name="Predicted Change in Price"))
fig.show()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0939
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.6097e-04
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 9.0472e-04
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 8.5234e-04
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0011
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.6102e-04
Epoch 7/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.2006e-04
Epoch 8/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 6.7557e-04
Epoch 9/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 8.1172e-04
Epoch 10/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 7.8662e-04
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step
Epoch 1/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.1040
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.9972e-04
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 9.6785e-04
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/st